In [1]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

/Users/akshatpant/anaconda3/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/Users/akshatpant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/akshatpant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/akshatpant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/akshatpant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falli

In [14]:
data_linear = pd.read_csv('../Data/Mice Data/data_after_mice.csv')
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,annual_inc,issue_month,issue_year,dti,...,loan_status_Does_not_meet_the_credit_policy_Status_Charged_Off,loan_status_Does_not_meet_the_credit_policy_Status_Fully_Paid,loan_status_Fully_Paid,loan_status_In_Grace_Period,loan_status_Issued,loan_status_Late_16_30_days,loan_status_Late_31_120_days,pymnt_plan_y,initial_list_status_w,int_rate
0,0.119293,0.119293,0.118696,0.000859,0.003886,0.000239,0.572605,0.000286,0.047980,0.000660,...,0,0,1,0,0,0,0,0,0,10.65
1,0.067323,0.067323,0.067323,0.001616,0.001611,0.000000,0.807871,0.000323,0.054154,0.000027,...,0,0,0,0,0,0,0,0,0,15.27
2,0.152472,0.152472,0.152472,0.002287,0.005357,0.000635,0.778372,0.000762,0.127759,0.000554,...,0,0,1,0,0,0,0,0,0,15.96
3,0.155753,0.155753,0.155753,0.000561,0.005285,0.000156,0.766305,0.000187,0.031322,0.000312,...,0,0,1,0,0,0,0,0,0,13.49
4,0.032231,0.032231,0.032231,0.000645,0.000728,0.000011,0.859494,0.000129,0.021606,0.000193,...,0,0,0,0,0,0,0,0,0,12.69


In [15]:
predictors = ['grade_C','grade_D','grade_E',
        'grade_F','grade_G','total_rec_int',
        'total_pymnt_inv','funded_amnt_inv','sub_grade_B5',
        'sub_grade_C5','sub_grade_C4','sub_grade_C3','sub_grade_B4','sub_grade_D5']

predictors

['grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'total_rec_int',
 'total_pymnt_inv',
 'funded_amnt_inv',
 'sub_grade_B5',
 'sub_grade_C5',
 'sub_grade_C4',
 'sub_grade_C3',
 'sub_grade_B4',
 'sub_grade_D5']

In [16]:
response = ['int_rate']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data[predictors], data[response],
                                                    train_size=0.6, test_size=0.4)


X_train = X_train.values
X_test = X_test.values
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [18]:
tpot_config = {'sklearn.tree.DecisionTreeClassifier': {},
              'sklearn.neural_network.MLPClassifier': {'hidden_layer_sizes': [50, 25], 
                                                       'activation': ['relu', 'tanh', 'logistic', 'identity'],
                                                      'batch_size': [10, 20, 40, 60, 80, 100], 
                                                       'solver': ['adam', 'sgd', 'lbfgs'], 'max_iter': [10, 50, 100],
                                                      'batch_size': [10, 20, 40, 60, 80, 100]},
              'sklearn.linear_model.LinearRegression': {}}


In [19]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, config_dict = tpot_config)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('/Users/akshatpant/Desktop/Rishika/tpot_lendingClub_pipeline_lite.py')

A Jupyter Widget

Generation 1 - Current best internal CV score: -1.5233035049539687
Generation 2 - Current best internal CV score: -1.5233035049539687
Generation 3 - Current best internal CV score: -1.5233035049539687
Generation 4 - Current best internal CV score: -1.5233027505419723
Generation 5 - Current best internal CV score: -1.5233027505419723

Best pipeline: LinearRegression(CombineDFs(CombineDFs(input_matrix, input_matrix), LinearRegression(input_matrix)))
-1.5258951979649398


#### TPOT writes the optimized pipeline to an external python file specified in the tpot.export() function.
#### The code is copied from that file and pasted below.

In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy

In [ ]:
# NOTE: Make sure that the class is labeled 'target' in the data file
#tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = train_test_split(features, 
#                                                                                         tpot_data['target'].values, random_state=None)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data[predictors], data[response],
                                                    train_size=0.6, test_size=0.4)


X_train = X_train.values
X_test = X_test.values
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [22]:
# Average CV score on the training set was:-1.5233027505419723
exported_pipeline = make_pipeline(
    make_union(
        make_union(
            FunctionTransformer(copy),
            FunctionTransformer(copy)
        ),
        StackingEstimator(estimator=LinearRegression())
    ),
    LinearRegression()
)

In [23]:
exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

In [24]:
mape = 0

for i,j in zip(y_test, results):
    mape += np.abs((i-j)/i)
    
mape = (mape*100)/y_test.shape[0]

mape

8.551144342590337